# Libraries

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import random as rd
import pandas as pd
import numpy as np
import pickle as pkl
from tqdm import tqdm
import os
import time
from collections import defaultdict
from scipy import stats
from itertools import permutations, product
from numpy.linalg import norm
import seaborn as sns
sns.set(style="darkgrid")
pd.set_option("display.precision", 4)

# Summary

#### Evaluation strategies
We have identified three evaluation strategies:
1. For every user in the **train set** rank **all items**. | Book paper, *eva_one*
2. For every user in the **test set** rank **only the items this user has rated in the test set**. | Music paper, *eva_two*
3. For every user in the **test set** rank **only the items this user has NOT rated in the train set**. | Movie(?) paper, *eva_three*

#### Algorithms

In [2]:
algo_names = ["UserKNN",
              "ItemKNN", 
              "UserKNN with means", 
              "BPR",
              "MF",
              "PMF",
              "NMF",
              "WMF",
              "HPF",
              "NeuMF",
              "VAECF"
              ] # in the order that they are in the model list

# Import results

### Additions

In [3]:
data = ["movies", "music","books"]
evaluation = ["eva_one", "eva_two", "eva_three"]
pop_notion = ["pop_one", "pop_two", "pop_three"]

In [4]:
aspects = [data, evaluation, pop_notion]

In [5]:
some_aspects = [data, evaluation]

In [6]:
results_location = "data/results/"

In [7]:
# pkl.load(open(results_location+"df_item_dist_cornac_"+data[2]+"_"+evaluation[0]+".csv","rb"))

In [8]:
c = pd.read_csv(results_location+"df_item_dist_cornac_"+data[2]+"_"+evaluation[0]+".csv", index_col=0)

In [9]:
c2 = pd.read_csv(results_location+"df_item_dist_cornac_"+data[2]+"_"+evaluation[1]+".csv", index_col=0)

In [10]:
stats.pearsonr(c2["count"], c2.HPF)

PearsonRResult(statistic=0.9173785262794707, pvalue=0.0)

In [11]:
stats.pearsonr(c2["count"], c2.NMF)

PearsonRResult(statistic=0.9124281116550262, pvalue=0.0)

In [12]:
stats.pearsonr(c["count"], c.NMF)

PearsonRResult(statistic=-0.03643678357239864, pvalue=0.0024315820834472663)

### Define aspect combinations

In [13]:
all_combinations = list(product(*aspects))
all_combinations = [x for x in all_combinations if not ("movies" in x)&("pop_three" in x)]
all_combinations = [x for x in all_combinations if not ("books" in x)&("pop_three" in x)]
# remove combinations of "pop_three" that dont include music.
some_combinations = list(product(*some_aspects))

In [14]:
sum_exists = 0
for combi in some_combinations:
    path = results_location+"df_item_dist_cornac_"+combi[0]+"_"+combi[1]+".csv"
    exists = os.path.exists(path)
    print(combi, exists)
    sum_exists+=exists
print(sum_exists, "out of", len(some_combinations))

('movies', 'eva_one') True
('movies', 'eva_two') True
('movies', 'eva_three') True
('music', 'eva_one') True
('music', 'eva_two') True
('music', 'eva_three') True
('books', 'eva_one') True
('books', 'eva_two') True
('books', 'eva_three') True
9 out of 9


In [18]:
new_results_location = 'results/'


In [21]:
sum_exists = 0

for combi in all_combinations:
    path = new_results_location+"low_gap_vals_"+combi[0]+"_"+combi[1]+"_"+combi[2]+".pickle"
    exists = os.path.exists(path)
    sum_exists+=exists
    print(combi, "|",exists)
print(sum_exists, "out of", len(all_combinations))

('movies', 'eva_one', 'pop_one') | True
('movies', 'eva_one', 'pop_two') | True
('movies', 'eva_two', 'pop_one') | True
('movies', 'eva_two', 'pop_two') | False
('movies', 'eva_three', 'pop_one') | False
('movies', 'eva_three', 'pop_two') | False
('music', 'eva_one', 'pop_one') | True
('music', 'eva_one', 'pop_two') | True
('music', 'eva_one', 'pop_three') | True
('music', 'eva_two', 'pop_one') | True
('music', 'eva_two', 'pop_two') | True
('music', 'eva_two', 'pop_three') | True
('music', 'eva_three', 'pop_one') | True
('music', 'eva_three', 'pop_two') | True
('music', 'eva_three', 'pop_three') | True
('books', 'eva_one', 'pop_one') | True
('books', 'eva_one', 'pop_two') | True
('books', 'eva_two', 'pop_one') | True
('books', 'eva_two', 'pop_two') | True
('books', 'eva_three', 'pop_one') | True
('books', 'eva_three', 'pop_two') | True
18 out of 21


# GAP

## TTESTS

In [58]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'music' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        #path_ttest = 'gap_ttests_books_eva_one_pop_one.pickle'
        path_ttest = new_results_location+"gap_ttests_music_"+eva_str+"_"+pop_n+".pickle"
        pvalues = pd.DataFrame(pkl.load(open(path_ttest,"rb") )[2:], index=algo_names, columns = ['low-med','low-high','med-high'])
        print((pvalues<0.005) & (pvalues>0))

eva_one pop_one
----------------
                    low-med  low-high  med-high
UserKNN                True      True      True
ItemKNN                True      True      True
UserKNN with means     True      True      True
BPR                    True      True      True
MF                     True     False      True
PMF                   False     False     False
NMF                   False     False     False
WMF                    True      True      True
HPF                    True      True      True
NeuMF                  True      True      True
VAECF                  True      True      True
eva_one pop_two
----------------
                    low-med  low-high  med-high
UserKNN                True      True     False
ItemKNN                True      True      True
UserKNN with means     True      True      True
BPR                    True      True      True
MF                     True     False      True
PMF                   False     False     False
NMF                   

In [22]:
combi = all_combinations[0]
path = new_results_location+"low_gap_vals_"+combi[0]+"_"+combi[1]+"_"+combi[2]+".pickle"


In [23]:
pop_gaps = []
for p in pop_notion:
    for v in ["low","med","high"]:
        pop_gaps.append(p+"_"+v)

In [24]:
pop_gaps

['pop_one_low',
 'pop_one_med',
 'pop_one_high',
 'pop_two_low',
 'pop_two_med',
 'pop_two_high',
 'pop_three_low',
 'pop_three_med',
 'pop_three_high']

### Music

In [25]:
music_GAPs_eva1 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)
music_GAPs_eva2 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)
music_GAPs_eva3 = pd.DataFrame(columns=pop_gaps, index = algo_names).fillna(0)

In [28]:
for combi in all_combinations:
    if "music" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        path_low = new_results_location+"low_gap_vals_music_"+eva_str+"_"+pop_n+".pickle"
        path_med = new_results_location+"med_gap_vals_music_"+eva_str+"_"+pop_n+".pickle"
        path_high = new_results_location+"high_gap_vals_music_"+eva_str+"_"+pop_n+".pickle"
        low_gap_vals = pd.DataFrame([pkl.load(open(path_low,"rb") )[2:]], columns=algo_names)
        med_gap_vals = pd.DataFrame([pkl.load(open(path_med,"rb") )[2:]], columns=algo_names)
        high_gap_vals = pd.DataFrame([pkl.load(open(path_high,"rb") )[2:]], columns=algo_names)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                music_GAPs_eva1.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                music_GAPs_eva1.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                music_GAPs_eva1.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                music_GAPs_eva2.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                music_GAPs_eva2.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                music_GAPs_eva2.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
            
        else:
            for algorithm in algo_names:
                music_GAPs_eva3.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                music_GAPs_eva3.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                music_GAPs_eva3.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]

In [29]:
np.round(music_GAPs_eva1,1)

,pop_one_low,pop_one_med,pop_one_high,pop_two_low,pop_two_med,pop_two_high,pop_three_low,pop_three_med,pop_three_high
UserKNN,-32.6,-32.6,-47.8,-27.6,-30.9,-52.7,-53.2,-28.2,-30.4
ItemKNN,-20.8,-13.4,-28.6,-17.0,-11.6,-33.3,-17.4,-13.6,-24.2
UserKNN with means,6.3,-6.4,-25.7,10.9,-4.7,-30.0,-34.7,-4.6,6.0
BPR,596.6,352.6,240.0,611.0,358.2,230.8,379.8,355.4,319.4
MF,-49.0,-68.9,-78.3,-47.1,-68.6,-79.1,-67.6,-68.1,-71.4
PMF,-69.7,-81.5,-87.1,-68.6,-81.3,-87.6,-80.8,-81.1,-83.1
NMF,-78.6,-87.1,-90.9,-77.8,-87.0,-91.1,-86.1,-86.8,-88.4
WMF,-2.8,14.2,1.2,0.6,17.8,-7.0,-30.4,20.0,32.5
HPF,150.4,143.8,125.2,166.0,148.6,110.1,93.2,153.2,167.6
NeuMF,399.1,242.3,157.3,399.6,243.9,159.2,262.2,239.0,213.4


In [62]:
music_GAPs_eva1.columns =["eva_one_"+x for x in music_GAPs_eva1.columns]
music_GAPs_eva2.columns =["eva_two_"+x for x in music_GAPs_eva2.columns]
music_GAPs_eva3.columns =["eva_three_"+x for x in music_GAPs_eva3.columns]

In [63]:
combined_music_GAPs = pd.concat([music_GAPs_eva1, music_GAPs_eva2, music_GAPs_eva3], axis=1)

In [65]:
combined_music_GAPs = np.round(combined_music_GAPs,1)

In [67]:
with open("latex_tables/music_GAPs_combined.tex", "w") as tf:
    tf.write(combined_music_GAPs.to_latex())

In [69]:
with open("latex_tables/music_GAPs_eva1.tex", "w") as tf:
    tf.write(np.round(music_GAPs_eva1,1).to_latex())
with open("latex_tables/music_GAPs_eva2.tex", "w") as tf:
    tf.write(np.round(music_GAPs_eva2,1).to_latex())
with open("latex_tables/music_GAPs_eva3.tex", "w") as tf:
    tf.write(np.round(music_GAPs_eva3,1).to_latex())

### Books

#### ttest?

In [50]:
new_results_location = 'results/'
for combi in all_combinations:
    
    if 'books' in combi: 
        eva_str = combi[1]
        pop_n = combi[2]
        print(eva_str, pop_n)
        print('----------------')
        #path_ttest = 'gap_ttests_books_eva_one_pop_one.pickle'
        path_ttest = new_results_location+"gap_ttests_books_"+eva_str+"_"+pop_n+".pickle"
        pvalues = pd.DataFrame(pkl.load(open(path_ttest,"rb") )[2:], index=algo_names, columns = ['low-med','low-high','med-high'])
        print(pvalues)

eva_one pop_one
----------------
                       low-med     low-high     med-high
UserKNN             4.0165e-68   5.2098e-86   1.4221e-07
ItemKNN             3.1596e-18   4.1252e-22   3.2855e-03
UserKNN with means  1.5172e-68   2.2137e-83   4.1643e-07
BPR                 1.0000e+00   1.0110e-53  1.2803e-113
MF                  4.7744e-18   5.1027e-02   4.2234e-09
PMF                 5.6429e-01   3.3088e-01   8.0106e-02
NMF                 6.5556e-01   4.4229e-01   6.1823e-01
WMF                 0.0000e+00   0.0000e+00   3.6785e-72
HPF                 3.0099e-83  1.9227e-149   3.8681e-43
NeuMF               0.0000e+00  1.5682e-193   0.0000e+00
VAECF               4.1089e-99  8.5488e-208   7.1493e-75
eva_one pop_two
----------------
                        low-med     low-high     med-high
UserKNN             3.7053e-101  2.0852e-158   1.2226e-46
ItemKNN              2.3881e-27   2.3209e-70   1.2995e-27
UserKNN with means  1.8509e-109  2.3763e-180   5.4732e-57
BPR               

#### gap

In [22]:
books_GAPs_eva1 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
books_GAPs_eva2 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
books_GAPs_eva3 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
for combi in all_combinations:
    if "books" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        path_low = results_location+"low_gap_vals_books_"+eva_str+"_"+pop_n+".pickle"
        path_med = results_location+"med_gap_vals_books_"+eva_str+"_"+pop_n+".pickle"
        path_high = results_location+"high_gap_vals_books_"+eva_str+"_"+pop_n+".pickle"
        low_gap_vals = pd.DataFrame([pkl.load(open(path_low,"rb") )[2:]], columns=algo_names)
        med_gap_vals = pd.DataFrame([pkl.load(open(path_med,"rb") )[2:]], columns=algo_names)
        high_gap_vals = pd.DataFrame([pkl.load(open(path_high,"rb") )[2:]], columns=algo_names)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                books_GAPs_eva1.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                books_GAPs_eva1.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                books_GAPs_eva1.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                books_GAPs_eva2.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                books_GAPs_eva2.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                books_GAPs_eva2.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
            
        else:
            for algorithm in algo_names:
                books_GAPs_eva3.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                books_GAPs_eva3.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                books_GAPs_eva3.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]

In [23]:
books_GAPs_eva1

,pop_one_low,pop_one_med,pop_one_high,pop_two_low,pop_two_med,pop_two_high
UserKNN,25.2487,-54.5058,-73.2624,70.9276,-51.1751,-80.3198
ItemKNN,56.1647,-34.4437,-61.3080,109.4333,-28.1203,-72.4008
UserKNN with means,22.1253,-54.9305,-73.4937,67.2140,-51.4353,-80.8482
BPR,1118.9694,523.8554,295.2193,1434.7379,572.2700,228.3943
MF,22.9898,-23.8985,-57.7958,44.6767,-18.2631,-62.3604
PMF,186.1511,44.9508,-5.2485,252.9929,53.0784,-15.1389
NMF,-47.1272,-72.8368,-82.7174,-32.9785,-70.8432,-85.5686
WMF,39.0438,109.9239,81.4392,49.0890,118.9020,67.0285
HPF,283.5865,155.3489,91.2869,369.6912,168.3932,71.7007
NeuMF,1030.6557,478.6574,266.5858,1323.5469,523.5644,204.6023


In [24]:
books_GAPs_eva1.columns =["eva_one_"+x for x in books_GAPs_eva1.columns]
books_GAPs_eva2.columns =["eva_two_"+x for x in books_GAPs_eva2.columns]
books_GAPs_eva3.columns =["eva_three_"+x for x in books_GAPs_eva3.columns]

In [25]:
books_GAPs_eva1

,eva_one_pop_one_low,eva_one_pop_one_med,eva_one_pop_one_high,eva_one_pop_two_low,eva_one_pop_two_med,eva_one_pop_two_high
UserKNN,25.2487,-54.5058,-73.2624,70.9276,-51.1751,-80.3198
ItemKNN,56.1647,-34.4437,-61.3080,109.4333,-28.1203,-72.4008
UserKNN with means,22.1253,-54.9305,-73.4937,67.2140,-51.4353,-80.8482
BPR,1118.9694,523.8554,295.2193,1434.7379,572.2700,228.3943
MF,22.9898,-23.8985,-57.7958,44.6767,-18.2631,-62.3604
PMF,186.1511,44.9508,-5.2485,252.9929,53.0784,-15.1389
NMF,-47.1272,-72.8368,-82.7174,-32.9785,-70.8432,-85.5686
WMF,39.0438,109.9239,81.4392,49.0890,118.9020,67.0285
HPF,283.5865,155.3489,91.2869,369.6912,168.3932,71.7007
NeuMF,1030.6557,478.6574,266.5858,1323.5469,523.5644,204.6023


In [26]:
with open("latex_tables/books_GAPs_eva1.tex", "w") as tf:
    tf.write(np.round(books_GAPs_eva1,1).to_latex())
with open("latex_tables/books_GAPs_eva2.tex", "w") as tf:
    tf.write(np.round(books_GAPs_eva2,1).to_latex())
with open("latex_tables/books_GAPs_eva3.tex", "w") as tf:
    tf.write(np.round(books_GAPs_eva3,1).to_latex())

### Movies

In [92]:
movies_GAPs_eva1 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
movies_GAPs_eva2 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
movies_GAPs_eva3 = pd.DataFrame(columns=pop_gaps[:-3], index = algo_names).fillna(0)
for combi in all_combinations:
    if "movies" in combi:
        eva_str = combi[1]
        pop_n = combi[2]
        path_low = results_location+"low_gap_vals_movies_"+eva_str+"_"+pop_n+".pickle"
        path_med = results_location+"med_gap_vals_movies_"+eva_str+"_"+pop_n+".pickle"
        path_high = results_location+"high_gap_vals_movies_"+eva_str+"_"+pop_n+".pickle"
        low_gap_vals = pd.DataFrame([pkl.load(open(path_low,"rb") )[2:]], columns=algo_names)
        med_gap_vals = pd.DataFrame([pkl.load(open(path_med,"rb") )[2:]], columns=algo_names)
        high_gap_vals = pd.DataFrame([pkl.load(open(path_high,"rb") )[2:]], columns=algo_names)
        
        if eva_str=="eva_one":
            for algorithm in algo_names:
                movies_GAPs_eva1.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                movies_GAPs_eva1.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                movies_GAPs_eva1.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
                        
        elif eva_str=="eva_two":
            for algorithm in algo_names:
                movies_GAPs_eva2.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                movies_GAPs_eva2.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                movies_GAPs_eva2.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]
            
        else:
            for algorithm in algo_names:
                movies_GAPs_eva3.at[algorithm, pop_n+"_low"] = low_gap_vals[algorithm][0]
                movies_GAPs_eva3.at[algorithm, pop_n+"_med"] = med_gap_vals[algorithm][0]
                movies_GAPs_eva3.at[algorithm, pop_n+"_high"] = high_gap_vals[algorithm][0]

In [93]:
movies_GAPs_eva1.columns =["eva_one_"+x for x in movies_GAPs_eva1.columns]
movies_GAPs_eva2.columns =["eva_two_"+x for x in movies_GAPs_eva2.columns]
movies_GAPs_eva3.columns =["eva_three_"+x for x in movies_GAPs_eva3.columns]

In [94]:
movies_GAPs_eva3

,eva_three_pop_one_low,eva_three_pop_one_med,eva_three_pop_one_high,eva_three_pop_two_low,eva_three_pop_two_med,eva_three_pop_two_high
UserKNN,-99.5484,-99.6872,-99.7602,-99.5407,-99.6832,-99.7681
ItemKNN,-60.6197,-82.9439,-97.1015,-52.4235,-83.7445,-98.5698
UserKNN with means,-99.5799,-99.7152,-99.7901,-99.5696,-99.7112,-99.7992
BPR,269.9329,170.1527,108.9213,274.3657,174.2339,101.4751
MF,-45.1403,-62.9024,-70.7027,-42.2919,-62.4925,-72.4650
PMF,45.5115,16.5913,2.9640,45.4986,15.4986,-6.5877
NMF,-93.0218,-95.2554,-96.1663,-93.0138,-95.1441,-96.3484
WMF,215.6487,129.4689,79.2923,220.3063,133.1679,71.9845
HPF,104.3257,82.9136,59.9872,108.7818,83.4322,57.7976
NeuMF,187.8243,133.4493,93.5356,191.2450,136.0481,88.4178


In [95]:
with open("latex_tables/movies_GAPs_eva1.tex", "w") as tf:
    tf.write(np.round(movies_GAPs_eva1,1).to_latex())
with open("latex_tables/movies_GAPs_eva2.tex", "w") as tf:
    tf.write(np.round(movies_GAPs_eva2,1).to_latex())
with open("latex_tables/movies_GAPs_eva3.tex", "w") as tf:
    tf.write(np.round(movies_GAPs_eva3,1).to_latex())